In [6]:
import os
os.chdir("../system")

In [9]:
import visualize
import image_observers.YOLOv4.detector as yolo
from pathlib import Path
import importlib as imp
import pandas as pd

In [3]:
imp.reload(visualize)

<module 'visualize' from '/home/tal/dev/reptilearn/system/visualize.py'>

In [3]:
vid_path = Path('/data/reptilearn/experiments/yolo_first_one_20210419-152027/top_20210419-152613.mp4')

In [4]:
det = yolo.YOLOv4Detector()
det.load()

YOLOv4 detector loaded successfully (416x416; image_observers/YOLOv4/yolo4_2306.cfg).


In [5]:
visualize.process_video(vid_path, "../analysis/test_yolo.mp4", [
    visualize.online_bbox_visualizer
])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
head_bbox_path = Path("/data/reptilearn/experiments/light_exp_try2_20210421-143906/head_bbox.csv")

In [16]:
head_bbox_df = pd.read_csv(str(head_bbox_path))

In [20]:
head_bbox_df[head_bbox_df['x1'].isna()].shape

(9251, 6)

In [21]:
head_bbox_df.shape

(92862, 6)